# 🎙️ Generate TTS Audio for Voice Lectures

Tạo audio cho các đoạn `<teacher_script>` trong file markdown.

**Bước 1:** Upload file markdown
**Bước 2:** Chạy các cell
**Bước 3:** Download file zip chứa audio

In [ ]:
# Install edge-tts
!pip install edge-tts -q
print("✅ Installed edge-tts")

In [ ]:
# Upload markdown file
from google.colab import files
import os

print("📁 Upload file markdown của bạn:")
uploaded = files.upload()

MD_FILE = list(uploaded.keys())[0]
print(f"\n✅ Đã upload: {MD_FILE}")

In [ ]:
# Parse teacher scripts
import re
import hashlib

with open(MD_FILE, 'r', encoding='utf-8') as f:
    content = f.read()

pattern = r'<teacher_script([^>]*)>([\s\S]*?)</teacher_script>'
matches = re.findall(pattern, content, re.IGNORECASE)

scripts = []
for i, (attrs, text) in enumerate(matches):
    text = text.strip()
    if text:
        text_hash = hashlib.md5(text.encode()).hexdigest()[:8]
        scripts.append({
            'index': i,
            'text': text,
            'hash': text_hash,
            'filename': f'ts_{i:03d}_{text_hash}.mp3'
        })

print(f"📝 Tìm thấy {len(scripts)} teacher scripts:\n")
for s in scripts[:5]:
    print(f"  {s['index']+1}. {s['text'][:50]}...")
if len(scripts) > 5:
    print(f"  ... và {len(scripts)-5} scripts nữa")

In [ ]:
# Generate audio files
import edge_tts
import asyncio

VOICE = "vi-VN-HoaiMyNeural"  # Female Vietnamese voice
# VOICE = "vi-VN-NamMinhNeural"  # Male Vietnamese voice

OUTPUT_DIR = "audio"
os.makedirs(OUTPUT_DIR, exist_ok=True)

async def generate_audio(text, output_path):
    communicate = edge_tts.Communicate(text, VOICE)
    await communicate.save(output_path)

async def generate_all():
    for s in scripts:
        output_path = os.path.join(OUTPUT_DIR, s['filename'])
        print(f"🔊 [{s['index']+1}/{len(scripts)}] {s['filename']}")
        try:
            await generate_audio(s['text'], output_path)
            print(f"   ✅ OK")
        except Exception as e:
            print(f"   ❌ Error: {e}")

await generate_all()
print(f"\n✅ Đã tạo {len(os.listdir(OUTPUT_DIR))} file audio")

In [ ]:
# Create updated markdown with href attributes
updated_content = content

for s in scripts:
    # Find and update the teacher_script tag
    old_pattern = f'<teacher_script([^>]*)>{re.escape(s["text"])}</teacher_script>'
    
    def replace_func(match):
        attrs = match.group(1)
        if 'href=' in attrs:
            attrs = re.sub(r'href="[^"]*"', f'href="audio/{s["filename"]}"', attrs)
        else:
            attrs = f'{attrs} href="audio/{s["filename"]}"'.strip()
        return f'<teacher_script {attrs}>{s["text"]}</teacher_script>'
    
    updated_content = re.sub(old_pattern, replace_func, updated_content, flags=re.IGNORECASE)

# Save updated markdown
updated_filename = MD_FILE.replace('.md', '_with_audio.md')
with open(updated_filename, 'w', encoding='utf-8') as f:
    f.write(updated_content)

print(f"✅ Đã tạo file markdown mới: {updated_filename}")

In [ ]:
# Download everything as zip
import shutil

# Create zip
zip_name = MD_FILE.replace('.md', '_audio')
shutil.make_archive(zip_name, 'zip', '.', OUTPUT_DIR)

# Also add the updated markdown to a combined zip
combined_dir = 'output'
os.makedirs(combined_dir, exist_ok=True)
shutil.copytree(OUTPUT_DIR, os.path.join(combined_dir, 'audio'), dirs_exist_ok=True)
shutil.copy(updated_filename, combined_dir)
shutil.make_archive('voice_lecture_output', 'zip', combined_dir)

print("📦 Download files:")
files.download('voice_lecture_output.zip')

## 📋 Hướng dẫn sử dụng

1. Giải nén file zip
2. Copy thư mục `audio/` vào cùng thư mục với file markdown gốc
3. Thay file markdown gốc bằng file `*_with_audio.md`
4. Hoặc copy các thuộc tính `href="audio/..."` vào file gốc

**Cấu trúc file:**
```
unit-07/
├── getting-started.md      # File gốc (updated với href)
└── audio/
    ├── ts_000_abc123.mp3
    ├── ts_001_def456.mp3
    └── ...
```